## A simple model for demand and supply of publicly-provided services in a city

In [1]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [2]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [3]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [4]:
from src.models.services_supply import ServiceUnit, ServiceEvaluator, UnitFactory, SchoolFactory, LibraryFactory 
from src.models.services_supply import make_shapely_point, get_random_pos #TODO refactor this fun into common utils

In [5]:
quicktest = [ServiceUnit(ServiceType.PoliceStation, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]

ServiceEvaluator(quicktest).evaluate_services_at(get_random_pos(4))
del quicktest

In [6]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)

Location data found


In [7]:
## Load scuole
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

Location data found


In [60]:
class MappedPositionFrame(pd.DataFrame):
    '''A class to collect an array of position alongside areas labels'''
    # do not override DataFrame __init__
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self['Positions'] = list(map(lambda x,y: geopy.Point(y,x), 
                                     self[common_cfg.coordColNames[0]], self[common_cfg.coordColNames[1]]))
    @property
    def positions(self):
        return None

In [56]:
class GridMaker():
    '''
    A class to create a grid and map it to various given land subdivisions
    '''
    def __init__(self, cityGeoFilesDict, gridStep=0.1): #gridStep in km
        self.quartiereKey = 'quartieri' #hardcoded
        self.IDquartiereCol = common_cfg.IdQuartiereColName
        
        # load division geofiles with geopandas
        self.subdivisions = {}
        for name, fileName in cityGeoFilesDict.items():
            self.subdivisions[name] = gpd.read_file(fileName)
        
        # compute city boundary
        self.cityBoundary = shapely.ops.cascaded_union(self.subdivisions[self.quartiereKey]['geometry'])
        
        # precompute coordinate ranges
        self.longRange = (self.cityBoundary.bounds[0], self.cityBoundary.bounds[2])
        self.latRange = (self.cityBoundary.bounds[1], self.cityBoundary.bounds[3])
        self.longNGrid = int(self.longitudeRangeKm // gridStep)+1
        self.latNGrid = int(self.latitudeRangeKm // gridStep)+1
        
        (self.xPlot, self.yPlot) = np.meshgrid(np.linspace(*self.longRange, self.longNGrid),
                                     np.linspace(*self.latRange, self.latNGrid), indexing='ij')
        
        # construct point objects with same shape
        self.bInPerimeter = np.empty_like(self.xPlot, dtype=bool)
        self.IDquartiere = np.empty_like(self.xPlot, dtype=object)
        
        for (i,j),_ in np.ndenumerate(self.bInPerimeter):
            shplyPoint = shapely.geometry.Point((self.xPlot[i,j], self.yPlot[i,j]))
            # mark points outside boundaries
            self.bInPerimeter[i,j] = self.cityBoundary.contains(shplyPoint)
            if self.bInPerimeter[i,j]:
                bFound = False
                for iRow in range(self.subdivisions[self.quartiereKey].shape[0]):
                    thisQuartierePolygon = self.subdivisions[self.quartiereKey]['geometry'][iRow]
                    if thisQuartierePolygon.contains(shplyPoint):
                        # assign found ID
                        self.IDquartiere[i,j] = self.subdivisions[
                            self.quartiereKey][self.IDquartiereCol][iRow]
                        bFound = True
                        break # skip remanining zones
                assert bFound, 'Point within city boundary was not assigned to any zone'
        
        # initialise mapping dict
        mappingDict = {
            common_cfg.coordColNames[0]:self.xPlot[self.bInPerimeter].flatten(), #long
            common_cfg.coordColNames[1]:self.yPlot[self.bInPerimeter].flatten(), #lat
            self.IDquartiereCol: self.IDquartiere[self.bInPerimeter],       #quartiere aggregation
                        } 
        # call common format constructor
        self.mappedPositions = MappedPositionFrame(mappingDict)
        
    @property
    def longitudeRangeKm(self):
        return geopy.distance.great_circle(
            (self.latRange[0], self.longRange[0]), (self.latRange[0], self.longRange[1])).km
    @property
    def latitudeRangeKm(self):
        return geopy.distance.great_circle(
            (self.latRange[0], self.longRange[0]), (self.latRange[1], self.longRange[0])).km    

In [61]:
testgridmaking = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.5)

In [ ]:
## Aggregation
class UnitAggregator:
    '''
    A class to evaluate services on a grid and get average values for given land subdivisions
    '''
    def __init__(self, unitList, cityGeoFilesDict, IDQuartiereField, gridStep=0.1): #gridStep in km
        assert isinstance(unitList, list), 'List expected'
        assert all([isinstance(t, ServiceUnit) for t in unitList]), 'ServiceUnits expected in list'
        self.units = unitList
        self.quartiereKey = 'quartieri' #hardcoded
        self.IDquartiereCol = IDQuartiereField
        
        # load division geofiles with geopandas
        self.subdivisions = {}
        for name, fileName in cityGeoFilesDict.items():
            self.subdivisions[name] = gpd.read_file(fileName)
        
        # compute city boundary
        self.cityBoundary = shapely.ops.cascaded_union(self.subdivisions[self.quartiereKey]['geometry'])
        
        # precompute coordinate rangesquartiereKey
        self.longRange = (self.cityBoundary.bounds[0], self.cityBoundary.bounds[2])
        self.latRange = (self.cityBoundary.bounds[1], self.cityBoundary.bounds[3])
        self.longNGrid = int(self.longitudeRangeKm // gridStep)+1
        self.latNGrid = int(self.latitudeRangeKm // gridStep)+1
        
        # precompute plotting grid
        (self.xPlot, self.yPlot) = np.meshgrid(np.linspace(*self.longRange, self.longNGrid),
                                               np.linspace(*self.latRange, self.latNGrid),
                                        indexing='ij')
        
        # construct point objects with same shape
        self.plotPoints = np.empty_like(self.xPlot, dtype=object)
        self.bInPerimeter = np.empty_like(self.xPlot, dtype=bool)
        self.IDquartiere = np.empty_like(self.xPlot, dtype=object)
        
        for (i,j),_ in np.ndenumerate(self.plotPoints):
            self.plotPoints[i,j] = geopy.Point(self.yPlot[i,j], self.xPlot[i,j]) # (lat, long) format
            shplyPoint = make_shapely_point(self.plotPoints[i,j])
            # mark points outside boundaries
            self.bInPerimeter[i,j] = self.cityBoundary.contains(shplyPoint)
            if self.bInPerimeter[i,j]:
                bFound = False
                for iRow in range(self.subdivisions[self.quartiereKey].shape[0]):
                    thisQuartierePolygon = self.subdivisions[self.quartiereKey]['geometry'][iRow]
                    if thisQuartierePolygon.contains(shplyPoint):
                        # assign found ID
                        self.IDquartiere[i,j] = self.subdivisions[
                            self.quartiereKey][self.IDquartiereCol][iRow]
                        bFound = True
                        break # skip remanining zones
                assert bFound, 'Point within city boundary was not assigned to any zone'
        
    
    @property
    def longitude(self):
        return [unit.site.longitude for unit in self.units]
    @property
    def longitudeRangeKm(self):
        return geopy.distance.great_circle(
            (self.latRange[0], self.longRange[0]), (self.latRange[0], self.longRange[1])).km
    @property
    def latitudeRangeKm(self):
        return geopy.distance.great_circle(
            (self.latRange[0], self.longRange[0]), (self.latRange[1], self.longRange[0])).km
    @property
    def latitude(self):
        return [unit.site.latitude for unit in self.units]
    @property
    def scale(self):
        scales = [unit.scale for unit in self.units]
        return scales
    
    def compute_service_levels(self, servType):
        '''
        Evaluates on a grid the aggregate service level for the loaded ServiceUnits of a selected type.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        # extend internal cache with computed values
        self.serviceValues[servType] = evaluate_services_at(
            self.plotPoints.reshape(-1), self.units, [servType])[servType]
        return None
    
    @property
    def aggregate_values(self):
        
        assert self.serviceValues, 'Empty values on grid found, compute first please!'
        
        quartieri = self.subdivisions[self.quartiereKey][self.IDquartiereCol].values
        
        # initialize output
        averagedValues = {service: pd.DataFrame(np.zeros([len(quartieri), len(AgeGroup.all())]), 
                                 index=quartieri, columns=AgeGroup.all()) 
                 for service in self.serviceValues.keys()}
      
        leftOutZones = []
        for quartiere in quartieri:
            bThisQuartiere = (self.IDquartiere == quartiere)
            # loop over service types
            if not bThisQuartiere.any():
                print('Quartiere %s has no points in grid within it' % quartiere)
                leftOutZones.append(quartiere)
                continue
                
            for servType in self.serviceValues.keys():
                for ageGroup, valuesSeries in self.serviceValues[servType].items(): 
                    quartiereValues = [valuesSeries[tuple(p)] for p in self.plotPoints[bThisQuartiere]]
                    averagedValues[servType].loc[quartiere, ageGroup] = np.mean(quartiereValues)
                    
        return averagedValues, leftOutZones

In [ ]:
# Prepare service level aggregation by quartiere
schoolAggr = UnitAggregator(schoolUnits, 
                {'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'},
                IDQuartiereField = common_cfg.IdQuartiereColName,
                gridStep=.5)

In [ ]:
# Prepare service level aggregation by quartiere
libAggr = UnitAggregator(libraryUnits, 
                {'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'},
                IDQuartiereField = common_cfg.IdQuartiereColName,
                gridStep=.5)

In [ ]:
## Plot tools
from descartes import PolygonPatch

class unitPlotter:
    '''
    A class that plots various types of output from a UnitAggregator
    '''
    def __init__(self, unitAggregatorIn):
        assert isinstance(unitAggregatorIn, UnitAggregator), 'UnitAggregator class expected'
        self.ua = unitAggregatorIn
        
    def plot_locations(self):
        '''
        Plots the loaded ServiceUnits according to their locations and rescale them by the relative sizes
        '''
        plotScales = self.ua.scale/np.mean(self.ua.scale)
        plt.figure()
        plt.scatter(self.ua.longitude, self.ua.latitude, s=plotScales)
        plt.axis('equal')
        plt.show()
        return None
        
    def plot_grid_and_boundary(self):
        fig = plt.figure()
        ax = fig.gca()
        plt.scatter(self.ua.xPlot[self.ua.bInPerimeter], self.ua.yPlot[self.ua.bInPerimeter], cmap='k', s=1)
        ax.add_patch(PolygonPatch(self.ua.cityBoundary, fc='None',zorder=5))
        plt.axis('equal')
        plt.show()
        return None
        
    def plot_service_levels(self, servType):
        '''
        Plots a contour graph of the results for each ageGroup.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        for ageGroup, valuesSeries in self.ua.serviceValues[servType].items():
            valuesArray = np.vectorize(lambda p: valuesSeries[tuple(p)])(self.ua.plotPoints)
            if np.count_nonzero(valuesArray) > 0:
                plt.figure()
                plt.title(ageGroup)
                CS = plt.contourf(self.ua.xPlot, self.ua.yPlot, valuesArray, 40)
                cbar = plt.colorbar(CS)
                cbar.ax.set_ylabel('Service level')
                plt.show()
            
        return None

In [ ]:
plotterNew= unitPlotter(libAggr)
plotterNew.plot_grid_and_boundary()
#plotterNew.plot_service_levels(ServiceType.School)

In [ ]:
# Run evaluation
libAggr.compute_service_levels(ServiceType.Library)

In [ ]:
# Save and display aggregate values for zones
out, _ = schoolAggr.aggregate_values
finalSchoolData = out[ServiceType.Library].rename_axis(
    schoolAggr.IDquartiereCol)
finalSchoolData.to_csv('../data/output/Milano_librarySample.csv', encoding='utf-8')
finalSchoolData

In [ ]:
outtest = pd.read_csv('../data/output/Milano_librarySample.csv',
                     index_col=common_cfg.IdQuartiereColName)
outtest

In [ ]:
plotterNew.plot_service_levels(ServiceType.Library)